<h1>Stream Cipher</h1>
<br>
Note: Many diagrams and some other items will not display in static views (like GitHub). [View 'live' on MyBinder](https://mybinder.org/v2/gh/jinjagit/Cryptography/master?filepath=StreamCipher.ipynb)
<br>

**NOTE: The following code should be run before other code in this notebook** (imports necessary libraries):

In [ ]:
%matplotlib inline 
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interact,FloatSlider,IntSlider

print("libraries imported")  # <- (re)run until output prints; may take a few seconds to run.

<br>
Notes on Lecture 3, [Introduction to Cryptography](https://www.youtube.com/channel/UC1usFRN4LCMcfIV7UjHNuQg/videos), by Christof Paar, and Chapter 2, [Understanding Crpytography](https://www.amazon.com/Understanding-Cryptography-Christof-Paar-ebook/dp/B00HWUO98A), by Paar & Pletzl:

<center><img src=diagrams/CryptoTree.svg / ></center>

Note: GSM is used by all mobile phones, was developed in mid 1980s, and was first large-scale application of encrytpion. GSM uses a stream cipher to encrypt digital voice (audio) signals, before transmitting to base towers.

**Definition:** A stream cipher encrypts bits individually (as against a block cipher, which encodes blocks of bits), **by adding a bit from a <i>key stream</i> to a plaintext bit.**

Usually, stream ciphers use very simple encrytption and decryption methods.

Each bit $\Large{x_i}$ is encrypted by adding a secret key stream bit $\Large{s_i}$, modulo 2.

<center><img src=diagrams/asynchStream.svg / ></center>

Stream ciphers tend to be small and fast and are, therefore, relevant to applications that run in environments with relatively limited resources. In general, however, block ciphers are still used more frequenly for encrytping computer communications. Neither kind can claim better overall efficiency (of software = cycles; or hardware = fewer gates/smaller chip area).

**Encryption and Decryption:**

<i>The plaintext, ciphertext and key consist of individual bits;</i>

$x_i, y_i, s_i\in\{1, 0\}$

**Encryption:** $y_i = e_{s_i}(x_i)\equiv x_i + s_i\mod{2}$<br></br>
**Decryption:** $x_i = d_{s_i}(y_i)\equiv y_i + s_i\mod{2}$

  1. Encryption and decryption are same functions!
  2. Why can we use a simple modulo 2 addition as encrytpion?
  3. What is the nature of the stream bits $\large{s_i}$?

**Proof:** that decryption uses same algorithm as encryption, by substitution of $\large{y_i}$ in decryption algorithm with the encryption algorithm, $\large{x_i + s_i}$ and further reduction of the resultant expression, thus:<br>
<br>
$d_{s_i}(y_i)\equiv y_i + s_i\mod{2}$ <br>
$d_{s_i}(y_i)\equiv (x_i + s_i) + s_i\mod{2}$ <br>
$d_{s_i}(y_i)\equiv x_i + 2s_i\mod{2}$ <br>
$d_{s_i}(y_i)\equiv x_i + 0\mod{2}$ <br>
$d_{s_i}(y_i)\equiv x_i\mod{2} \;\blacksquare$

Thus, the reason that we can use simple modulo 2 addition as decryption, (as well as encryption), is that $x_i, y_i, s_i \in\{1, 0\}$, and $x_i, y_i, s_i \in Z 2$. In other words, because they are all bits and, therefore, can only have values of either $0$, or $1$.

**RULE:** Modulo 2 addition and subtraction are the same operation.

In [ ]:
# calculates a+b mod 2, and a-b mod 2

a = int(input("enter 1st integer: "))
b = int(input("enter 2nd integer: "))

c = (a + b)%2
d = (a - b)%2

print("%d + %d, modulo 2 = %d" % (a, b, c))
print("%d - %d, modulo 2 = %d" % (a, b, d))

Thus, $\;\;\;\;d_{s_i}(y_i)\equiv y_i + s_i\mod{2}\;\;\;\;$ and $\;\;\;\;d_{s_i}(y_i)\equiv y_i - s_i\mod{2}\;\;\;\;$ are equivalent.

Consider the truth table for x, s and y:

|        |  x  |  s  |  y = x + s  |  y = x - s  |
|--------|-----|-----|-------------|-------------|
| case 1 |  0  |  0  |      0      |      0      |
| case 2 |  0  |  1  |      1      |      1      |
| case 3 |  1  |  0  |      1      |      1      |
| case 4 |  1  |  1  |      0      |      0      |

The above represents an [XOR](https://en.wikipedia.org/wiki/XOR_gate) binary gate, executed using either addition or subtraction in modulo 2.<br>
An XOR gate implements an exclusive **or**; that is, a true output results if one, and only one, of the inputs to the gate is true.

Idea: To apply modulo 2 to any binary integer value, we need only read the value of the last bit, (0 or 1), which gives the modulo 2 value of the integer. Not a great insight, as really it is another way of saying that the nature of binary means the 'units' digit of any integer will equal the modulo 2 value. Reading a bit from memory is probably generally faster than doing modulo math on an integer. However, <i>I imagine</i> most decent math libraries / languages exploit this fact when performing modulo 2 math on an integer (but, I do not know this).

In [ ]:
# compare modulo 2 representation of an integer with the last digit of its binary representation

a = int(input("enter integer: "))

get_bin = lambda x: format(x, 'b') # useful function for converting to binary string

binary = get_bin(a)
lastDigit = binary[-1:]
mod2 = a%2

print("binary representation: " + binary)
print("last binary digit: " + lastDigit)
print("%d modulo 2 = %d" % (a, mod2))

<center><img src=diagrams/streamBasic.svg / ></center>

Considering the above truth table and its consequences, we can see that <i>if the probability of any stream bit $\large{s_i}$ being $1$ or $0$ is equal,</i> then the encoding of either $x = 0$, or $x = 1$, has an equal chance of giving $y = 0$, or $y = 1$, in either case.<br>
<br>
**Flipping bits:** Also, the value of the stream bit has a particular effect on the $x$ value provided: $s = 1$ will always flip the input bit; $0\,\to\,1$, or $1\,\to\,0$, whereas $s = 0$ will not change the input bit.<br>
  As there are two modulo 2 addition (XOR) operations in the encoding/decoding stream, each using $\large{s_i}$ to enccode $\large{x_i}$ and decode $\large{y_i}$, $\large{x_i}$ is eventually either flipped twice; if $\large{s_i}$ $=1$, or not at all; if $\large{s_i}$ $=0$.